In [1]:
%%capture
!pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu

In [2]:
print("terminado")

terminado


In [7]:
# Used to import data from local.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

In [75]:
df=pd.read_parquet("/content/datasetVideos.parquet")
df=df.reset_index()
df=df.reset_index()
df=df.drop(["index","level_0"],axis=1)
df=df.reset_index()
df=df.rename(columns={"index":"id"})
print(df.shape)
df.head()

(954, 6)


,id,start,end,text,url,name
0,0,0.00,9.00,"Muy buenos días, tardes o noches para el que ...",/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Crear-modificar-Productos
1,1,9.00,19.24,Yo soy Sergio Gamboa y les voy a compartir un...,/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Crear-modificar-Productos
2,2,19.24,24.36,productos para el cotizador de Hub Avisos.,/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Crear-modificar-Productos
3,3,24.36,31.04,Les decía que si yo quiero agregar o modifica...,/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Crear-modificar-Productos
4,4,31.04,33.64,cómo es ese buen procedimiento.,/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Crear-modificar-Productos


In [77]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('eduardofv/stsb-m-mt-es-distilbert-base-uncased')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

cuda:0


In [78]:
# Convert abstracts to vectors
embeddings = model.encode(df["text"].to_list(), show_progress_bar=True)
print(len(embeddings))

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

954


In [79]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df["id"].values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 954


In [80]:
def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level
    DistilBERT model and finds similar vectors using FAISS.
    
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I

In [89]:
user_query="Como se puede cambiar el precio de un producto"

In [90]:
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [122.85165405273438, 124.40603637695312, 142.25640869140625, 143.42776489257812, 144.1502685546875, 144.17100524902344, 144.22018432617188, 145.02749633789062, 146.40782165527344, 146.6608123779297]

MAG paper IDs: [4, 29, 693, 506, 265, 383, 448, 489, 861, 823]


In [91]:
resultados=I.flatten().tolist()

In [92]:
len(set(resultados))

10

In [93]:
pd.concat([df[df["id"]==idx] for idx in resultados])

,id,start,end,text,url,name
4,4,31.04,33.64,cómo es ese buen procedimiento.,/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Crear-modificar-Productos
29,29,148.16,160.36,"Como sale ese índice, uno va y crea la funció...",/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Crear-modificar-Productos
693,693,2158.00,2160.00,que efectivamente es un producto.,/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video
506,506,1577.00,1579.00,Esto es más sencillo.,/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video
265,265,91.00,96.00,"siempre recomendarle descargarlo, tenerlo com...",/content/drive/MyDrive/Projects/29.buscadorWhi...,Guia-Modificacion-Estetica
383,383,600.00,605.00,"Aquí como estamos en fase de desarrollo, el t...",/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video
448,448,1171.00,1199.00,"Como pueden ver, después de esos dos, hay un ...",/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video
489,489,1524.00,1527.00,"Como todos, el good result section,",/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video
861,861,2795.00,2798.00,Y así es como debe estar configurado un ítem ...,/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video
823,823,2606.00,2607.00,¿Cómo se calcula el precio?,/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video


In [56]:
df[df["id"] == resultados[0]]

,index,start,end,text,url,name
576,271,1786.0,1787.0,Tengo que cambiarlos.,/content/drive/MyDrive/Projects/29.buscadorWhi...,Deploy-guide-Video
